**Discrete Event simulation for Take Home 1 of 4DM40**

In [1]:
# =================================
# Imports
# =================================
from PyCh import *
from numpy import random

# =================================
# Simulation parameters
# =================================
ta            = 360.0                                        # mean inter-arrival time
ca            = 1.0                                          # coefficient of variation of inter-arrival time
EPTs          = [ (180.0, 1.0), (90.0, 1.0), (60.0, 1.0) ]   # list with (te,ce) of wip-dependent EPT-distributions
#
#
#
warmupcount   = 1000                                         # number of lots to initially ignore (warmup)
lotcount      = 100000                                       # number of lots to run through system (simulation length)
shiftduration = 360.0                                        # duration of shift in minutes


# ====================================================
# Functions and processes for simulation
#
# There should be no need for editing the code below
# ====================================================
def Model():
    """
    Defines and runs the discrete event simulation model
    """
    # define environment
    env = Environment()
    # define channels
    a = Channel(env)
    b = Channel(env)
    # define processes
    G = Generator(env,a)
    W = Workstation(env,a,b,EPTs)
    E = Exit(env,b)
    # run simulation environment
    env.run(until=E)
    
@process
def Generator(env, c_out):
    """
    Generator process:
    Generates lots with mean inter arrival time ta and coefficient of variation ca
    """
    dist = lambda: random.gamma(1/(ca*ca),ca*ca*ta)
    while True:
        x = env.time
        yield env.execute(c_out.send(x))
        delay = dist()
        yield env.timeout(delay)
        
@process
def Workstation(env, c_in, c_out, EPTs):
    """
    Workstation process:
    Consists of a Buffer and a Server.
    
    @param EPTs: List with (te,ce) pairs for wip-dependent EPT-distributions of Server
    @type  EPTs: C{list} of (C{real},C{real})
    
    """
    c_bm = Channel(env)
    B = Buffer(env, c_in, c_bm)
    S = Server(env, c_bm, c_out, EPTs)
    yield B
    yield S
    
@process
def Buffer(env, c_in, c_out):
    """
    Buffer process:
    Adds received lots in list xs, and sends out lots (if available) together with
    the current number of lots remaining in the buffer.
    
    """
    xs = []
    target = 0
    while True:
        sending = c_out.send((xs[0],len(xs)-1)) if len(xs)>0 else None
        receiving = c_in.receive()
        x = yield env.select(sending, receiving)
        if selected(receiving):
            xs = xs + [x]
        if selected(sending):
            xs = xs[1:]
    
@process
def Server(env, c_in, c_out, EPTs):
    """
    Server process:
    Receives lots together with the number of lots in buffer. Based on the latter, selects the EPT-distribution
    to use for determining the processing time, after which the processed lot is sent out.
    If the wip in the workstation is larger than the number of EPT-distributions available in the list, the last
    EPT-distribution is used.
    
    @param EPTs: List with (te,ce) pairs for wip-dependent EPT-distributions of Server
    @type  EPTs: C{list} of (C{real},C{real})
    
    """
    u = [lambda: random.gamma(1/(ca*ca),ca*ca*ta) for (ta,ca) in EPTs]
    maxindex=len(u)-1
    while True:
        (lot,index) = yield env.execute(c_in.receive())
        delay = u[min(index,maxindex)]()
        yield env.timeout(delay)
        yield env.execute(c_out.send(lot))
    
@process
def Exit(env, c_in):
    """
    Exit process:
    Collects all jobs leaving the system and communicates the cumulative production to the controller upon request.
    Also determines the mean flow time of jobs.
    
    """
    n = -warmupcount
    meanflowtime = 0.0
    while n<=lotcount:
        x = yield env.select(c_in.receive())
        n = n + 1
        if n>0:
            flowtime = env.now - x
            meanflowtime = (n-1)/n * meanflowtime + flowtime/n
    print(f"Mean flowtime (in minutes): {meanflowtime:f}; mean Throughput (in lots per 6h shift): {n*shiftduration/env.now:f}.")

    
# call Model with required parameters
print(f"Simulation started...")
Model()


PyCh version 2.2 imported successfully.
 
Simulation started...
Mean flowtime (in minutes): 72.307172; mean Throughput (in lots per 6h shift): 0.993713.
